## 加载知识库数据

首先，需要使用标准格式加载数据。也就是说，我们需要将文本切成小块，从而确保传入 LLM 模型的数据为一段段小的文本片段。

In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
model_config_path = os.path.abspath(os.path.join('../custom_llms/'))
sys.path.insert(0, module_path)
sys.path.insert(0, model_config_path)

from custom_llms import (
    ZhipuAIEmbeddings,
    Zhipuai_LLM,
    load_api
)
api_key = load_api()
model = Zhipuai_LLM(zhipuai_api_key=api_key)

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Milvus
from langchain.document_loaders import TextLoader

loader = TextLoader('../../docs/state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

接下来，我们需要将小块的文本片段转化为向量并存储在向量数据库中。以下示例代码使用 OpenAI 的模型和 Zilliz Cloud 向量数据库。

In [3]:
embeddings = ZhipuAIEmbeddings()

vector_db = Milvus.from_documents(
    docs,
    embeddings,
    connection_args={
        "host": "172.23.115.108",
        "port": "19530",
        "user": "",
        "password": "",
        "secure": False
    }
)

## 查询数据

加载数据后，可以在问答链（Chain）中使用这些数据，下述代码主要解决上文提到的“幻觉”问题。

使用similarity_search方法将查询语句转化为特征向量，然后在 Zilliz Cloud 中搜索相似向量，以及相关的文档内容。

In [4]:
query = "What did the president say about Ketanji Brown Jackson"
docs = vector_db.similarity_search(query)

运行 load_qa_chain获取最终答案。这是一个最通用的用于回答问题的接口，它加载一整个链，可以根据所有数据库中文本进行问答。以下示例代码使用 OpenAI 作为 LLM 模型。在运行时，QA Chain 接收input_documents和 question，将其作为输入。input_documents是与数据库中的query相关的文档。LLM 基于这些文档的内容和所提问的问题来组织答案。

In [5]:
from langchain.chains.question_answering import load_qa_chain

llm = model
chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=docs, question=query)

"The president said that Ketanji Brown Jackson is one of the nation's top legal minds and will continue Justice Breyer's legacy of excellence. He nominated her to serve on the United States Supreme Court."